In [1]:
from torch.utils.tensorboard import SummaryWriter   
import pandas as pd,os
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_predict, KFold

os.environ["CUDA_VISIBLE_DEVICES"] = "1"


In [2]:
class RNN(nn.Module):
    def __init__(self, class_n, Input_Size):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(
            input_size=Input_Size,
            hidden_size=128,
            num_layers=1,
            batch_first=True,
        )

        self.out = nn.Linear(128, class_n)
    def forward(self, x):
        r_out, (h_c, h_h) = self.rnn(x, None)
        out = self.out(r_out[:, -1, :])
        return out
    
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.normal_(m.weight.data, 0, 0.01)
                # m.weight.data.normal_(0,0.01)
                m.bias.data.zero_()

class MLP(nn.Module):
    def __init__(self, num_classes, input_size):
        super(MLP,self).__init__()
        self.linear1 = nn.Linear(in_features=input_size, out_features=1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.dt1 = nn.Dropout(0.25)
        self.linear2 = nn.Linear(in_features=1024, out_features=256)
        self.bn2 = nn.BatchNorm1d(256)
        self.dt2 = nn.Dropout(0.25)
        self.linear3 = nn.Linear(in_features=256, out_features= num_classes)
        
    def forward(self, x):
        x = self.bn1(self.linear1(x))
        x = F.relu(x)
        # x = self.dt1(x)
        x = self.bn2(self.linear2(x))
        x = F.relu(x)
        # x = self.dt2(x)
        x = self.linear3(x)
        x = F.softmax(x, dim=1)
        return x
        
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                torch.nn.init.normal_(m.weight.data, 0, 0.01)
                # m.weight.data.normal_(0,0.01)
                m.bias.data.zero_()

In [3]:
def label2value(ans, data, label_train):
    X = []
    y = []
    
    for idi,i in enumerate(ans):
        pop = False
        for idj, j in enumerate(label_train):
            if j==i:
                pop = True
                y.append(idj)
        if pop==True:
            X.append(data[idi])
    return np.array(X), np.array(y)

def dataloader(subg_n):
    subgroup_ = {'class55':10, 'class7':100, 'class8': 50}
    data_train, ans_train= np.load(f'./dataset4/X_train_class{subgroup_[subg_n]}_time.npy'), np.load(f'./dataset4/y_train_class{subgroup_[subg_n]}_time.npy', allow_pickle=True)
    label_train = np.unique(ans_train, return_counts=False,return_index=False,return_inverse=False)

    X_train, y_train = label2value(ans_train, data_train, label_train)
    # print(len(np.unique(y_train, return_counts=False,return_index=False,return_inverse=False)))
    # print(X_train.shape, y_train.shape)
    return X_train, y_train

In [4]:
class RNN(nn.Module):
    def __init__(self, class_n, Input_Size):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(
            input_size=Input_Size,
            hidden_size=128,
            num_layers=1,
            batch_first=True,
        )

        self.out = nn.Linear(128, class_n)
    def forward(self, x):
        r_out, (h_c, h_h) = self.rnn(x, None)
        out = self.out(r_out[:, -1, :])
        return out
    
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.normal_(m.weight.data, 0, 0.01)
                # m.weight.data.normal_(0,0.01)
                m.bias.data.zero_()

class MLP(nn.Module):
    def __init__(self, num_classes, input_size):
        super(MLP,self).__init__()
        self.linear1 = nn.Linear(in_features=input_size, out_features=1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.dt1 = nn.Dropout(0.25)
        self.linear2 = nn.Linear(in_features=1024, out_features=256)
        self.bn2 = nn.BatchNorm1d(256)
        self.dt2 = nn.Dropout(0.25)
        self.linear3 = nn.Linear(in_features=256, out_features= num_classes)
        
    def forward(self, x):
        x = self.bn1(self.linear1(x))
        x = F.relu(x)
        # x = self.dt1(x)
        x = self.bn2(self.linear2(x))
        x = F.relu(x)
        # x = self.dt2(x)
        x = self.linear3(x)
        x = F.softmax(x, dim=1)
        return x
        
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                torch.nn.init.normal_(m.weight.data, 0, 0.01)
                # m.weight.data.normal_(0,0.01)
                m.bias.data.zero_()

In [5]:
batch_size = 128
k_cv = KFold(n_splits=10, random_state=42, shuffle=True) 

def accuracy(predictions, labels):
    classes = torch.argmax(predictions, dim=1)
    return torch.mean((classes == labels).float())

def predict(idx, valid_index, model_weight):
    x_, y_ = X_train[valid_index], y_train[valid_index]
    valid_zip = TensorDataset(torch.tensor(x_), torch.tensor(y_))
    valid_loader = DataLoader(dataset=valid_zip, batch_size=batch_size, shuffle=False)
    # Starting trainer
    Input_Size = np.array(X_train).shape[1]
    checkpoint = torch.load('./pth/'+model_weight, map_location=torch.device('cpu'))
    if 'LSTM' in model_weight:
        model = RNN(max(y_train)+1, Input_Size).to('cpu')
    else:
        model = MLP(max(y_train)+1, Input_Size).to('cpu')
    model.load_state_dict(checkpoint['state_dict'])
    model.to('cpu')
    # Validation
    correct2=0
    with torch.no_grad():
        model.eval()
        for step, (b_x, b_y) in enumerate(valid_loader):
            if 'LSTM' in model_weight:
                b_x_, b_y_= (torch.unsqueeze(b_x, 1)).data.type(torch.FloatTensor).to('cpu'), b_y.to('cpu')
            else:
                b_x_, b_y_= b_x.float().to('cpu'), b_y.to('cpu')
            r_out = model(b_x_)
            correct2  = correct2 + accuracy(r_out, b_y_)

            torch.cuda.empty_cache()
    valid_acc = 100 * correct2 / len(valid_loader)
    return valid_acc.item()

md_type_stack= ['MLP', 'LSTM']
# md_type_stack= ['MLP']
subgroup_dataset = ['class55', 'class7', 'class8']
# subgroup_dataset = ['class55']
weight_lsit = sorted(os.listdir('./pth'))
for j in md_type_stack:
    for jj in subgroup_dataset:
        acc = []
        X_train, y_train = dataloader(jj)
        next_weight = [i for i in weight_lsit if j in i and jj in i]
        temp  = next_weight[1::]
        temp.append(next_weight[0])
        next_weight = temp
        for idx,(train_index, valid_index) in enumerate(k_cv.split(X_train)):
            acc.append(predict(idx, valid_index, next_weight[idx]))
        print('model: ',j ,'class: ' ,jj)
        print('Best Accuracy', max(acc), '%')

model:  MLP class:  class55
Best Accuracy 96.34878540039062 %
model:  MLP class:  class7
Best Accuracy 98.41889953613281 %
model:  MLP class:  class8
Best Accuracy 98.95833587646484 %
model:  LSTM class:  class55
Best Accuracy 73.99462127685547 %
model:  LSTM class:  class7
Best Accuracy 98.046875 %
model:  LSTM class:  class8
Best Accuracy 98.17708587646484 %
